<a href="https://colab.research.google.com/github/OptimizationExpert/Pyomo/blob/main/Project_Family_Advanced_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s assume that Rene and Leo are both heads of household, and, what a coincidence, both families include three girls and three boys. The youngest child in Leo’s family is a girl, and in Rene’s family, a little girl has just arrived. In other words, there is a girl in Rene’s family whose age is less than one year. Neither family includes any twins, nor any children closer in  age than a year. All the children are under age ten. In each family, the sum of the ages of the girls is equal to the sum of the ages of the boys; in fact, the sum of the squares of the ages of the girls is equal to the sum of the squares of the ages of the boys. The sum of the ages of all these children is 60.

# Question: What are the ages of the children in these two families? How many solutions does this problem have?

In [1]:
!pip install ortools
from ortools.sat.python import cp_model # CP-SAT solver
import numpy as np
from random import randint
import random
from math import ceil
import pandas as pd
import matplotlib.pyplot as plt # Data visualization
#plt.rcParams["figure.figsize"] = (15,10)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [18]:
from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for kk,v in self.__variables.items():
            print(f"{kk}={self.Value(v)}")
        print(self.__solution_count )

    def solution_count(self):
        return self.__solution_count


def SearchForAllSolutionsSampleSat():
    """Showcases calling the solver to search for all solutions."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    families = ['F1' , 'F2']
    Jens = ['b' , 'g']
    kids = [k for k in range(3)]

    U = {(f,j,k):model.NewIntVar(0, 10, f"kid_{f}_{j}_{k}") for f in families for j in Jens \
         for k in kids}
    US = {(f,j,k):model.NewIntVar(0, 100, f"kidS_{f}_{j}_{k}") for f in families for j in Jens \
         for k in kids}
    expressions = [v for v in U.values()]
    model.Add(sum(expressions)== 60)
    model.Add(U["F1",'g',0]== 0)

    for (f,j,k), v in U.items():
      model.AddMultiplicationEquality(US[f,j,k],[v,v])
      if k <2:
        model.Add(U[f,j,k]<=U[f,j,k+1])


    for k in kids:
      for j in Jens:
        model.Add(U["F2",'g',0]<= U["F2",j,k])

    for f in families:
      expressions_family = [v for (F,j,k),v in U.items() if F==f]
      model.AddAllDifferent(expressions_family)
      expressions_family_boys = [v for (F,j,k),v in U.items() if F==f and j =='b']
      expressions_family_girls = [v for (F,j,k),v in U.items() if F==f and j =='g']
      model.Add(sum(expressions_family_boys)==sum(expressions_family_girls))

      expressions_family_boys_square = [v for (F,j,k),v in US.items() if F==f and j =='b']
      expressions_family_girls_square = [v for (F,j,k),v in US.items() if F==f and j =='g']
      model.Add(sum(expressions_family_boys_square)==sum(expressions_family_girls_square))





    # Create the constraints.

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(U)
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True
    # Solve.
    status = solver.Solve(model, solution_printer)

    print(f"Status = {solver.StatusName(status)}")
    #print(f"Number of solutions found: {solution_printer.solution_count()}")


SearchForAllSolutionsSampleSat()

('F1', 'b', 0)=1
('F1', 'b', 1)=3
('F1', 'b', 2)=8
('F1', 'g', 0)=0
('F1', 'g', 1)=5
('F1', 'g', 2)=7
('F2', 'b', 0)=4
('F2', 'b', 1)=5
('F2', 'b', 2)=9
('F2', 'g', 0)=3
('F2', 'g', 1)=7
('F2', 'g', 2)=8
1
('F1', 'b', 0)=1
('F1', 'b', 1)=4
('F1', 'b', 2)=10
('F1', 'g', 0)=0
('F1', 'g', 1)=6
('F1', 'g', 2)=9
('F2', 'b', 0)=3
('F2', 'b', 1)=4
('F2', 'b', 2)=8
('F2', 'g', 0)=2
('F2', 'g', 1)=6
('F2', 'g', 2)=7
2
('F1', 'b', 0)=1
('F1', 'b', 1)=4
('F1', 'b', 2)=10
('F1', 'g', 0)=0
('F1', 'g', 1)=6
('F1', 'g', 2)=9
('F2', 'b', 0)=1
('F2', 'b', 1)=4
('F2', 'b', 2)=10
('F2', 'g', 0)=0
('F2', 'g', 1)=6
('F2', 'g', 2)=9
3
('F1', 'b', 0)=2
('F1', 'b', 1)=3
('F1', 'b', 2)=10
('F1', 'g', 0)=0
('F1', 'g', 1)=7
('F1', 'g', 2)=8
('F2', 'b', 0)=1
('F2', 'b', 1)=4
('F2', 'b', 2)=10
('F2', 'g', 0)=0
('F2', 'g', 1)=6
('F2', 'g', 2)=9
4
('F1', 'b', 0)=1
('F1', 'b', 1)=2
('F1', 'b', 2)=6
('F1', 'g', 0)=0
('F1', 'g', 1)=4
('F1', 'g', 2)=5
('F2', 'b', 0)=5
('F2', 'b', 1)=6
('F2', 'b', 2)=10
('F2', 'g', 0)=4
